<a href="https://colab.research.google.com/github/ishapugalia/Abstractive-Text-Summarization/blob/main/model_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import os
import pandas as pd
import re

In [3]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [4]:
import tensorflow as tf

In [9]:
data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Dataset csv/train.csv")


In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
d_articles=pd.DataFrame(data.iloc[:10000,[0,1]])
print(d_articles[:1])
d_highlights=pd.DataFrame(data.iloc[:10000,[0,2]])
print(d_highlights[:1])
print(d_articles)

                                         id  \
0  0001d1afc246a7964130f43ae940af6bc6c57f01   

                                             article  
0  By . Associated Press . PUBLISHED: . 14:11 EST...  
                                         id  \
0  0001d1afc246a7964130f43ae940af6bc6c57f01   

                                          highlights  
0  Bishop John Folda, of North Dakota, is taking ...  
                                            id  \
0     0001d1afc246a7964130f43ae940af6bc6c57f01   
1     0002095e55fcbd3a2f366d9bf92a95433dc305ef   
2     00027e965c8264c35cc1bc55556db388da82b07f   
3     0002c17436637c4fe1837c935c04de47adb18e9a   
4     0003ad6ef0c37534f80b55b4235108024b407f0b   
...                                        ...   
9995  1c5f649f94e8f20b6fe624a842302208b781edbc   
9996  1c5f8a16032bff0ac292e80d33d9c473323db7aa   
9997  1c6060b597d97f86d96e7df9ecfc29517f7bbae4   
9998  1c613f908ae6868f2d43cfe5ac75c5d35bfd734b   
9999  1c61b5bd8b09b4f3a26aa740a7f7e11814

In [11]:
def cleantext(text):
    text = str(text)
    text=text.split()
    words=[]
    for t in text:
        if t.isalpha():
            words.append(t)
    text=" ".join(words)
    text=text.lower()
    text=re.sub(r"what's","what is ",text)
    text=re.sub(r"it's","it is ",text)
    text=re.sub(r"\'ve"," have ",text)
    text=re.sub(r"i'm","i am ",text)
    text=re.sub(r"\'re"," are ",text)
    text=re.sub(r"n't"," not ",text)
    text=re.sub(r"\'d"," would ",text)
    text=re.sub(r"\'s","s",text)
    text=re.sub(r"\'ll"," will ",text)
    text=re.sub(r"can't"," cannot ",text)
    text=re.sub(r" e g "," eg ",text)
    text=re.sub(r"e-mail","email",text)
    text=re.sub(r"9\\/11"," 911 ",text)
    text=re.sub(r" u.s"," american ",text)
    text=re.sub(r" u.n"," united nations ",text)
    text=re.sub(r"\n"," ",text)
    text=re.sub(r":"," ",text)
    text=re.sub(r"-"," ",text)
    text=re.sub(r"\_"," ",text)
    text=re.sub(r"\d+"," ",text)
    text=re.sub(r"[$#@%&*!~?%{}()]"," ",text)
    
    return text

In [12]:
d_articles["article"]= d_articles["article"].apply(lambda x: cleantext(x))

In [13]:
d_highlights["highlights"]= d_highlights["highlights"].apply(lambda x: cleantext(x))

In [14]:
VOCAB_SIZE = 29999
tokenizer = Tokenizer(num_words=VOCAB_SIZE)
tokenizer.fit_on_texts(list(d_articles.article))
article_sequences = tokenizer.texts_to_sequences(list(d_articles.article))
art_word_index = tokenizer.word_index
len(art_word_index)

75952

In [15]:
art_vocab = {}
counter = 0
for word in art_word_index.keys():
    if art_word_index[word] == 0:
        print("found 0!")
        break
    if art_word_index[word] > VOCAB_SIZE:
        continue
    else:
        art_vocab[word] = art_word_index[word]
        counter += 1

In [16]:
tokenizer.fit_on_texts(d_highlights)
summary_sequences = tokenizer.texts_to_sequences(d_highlights)
sum_word_index = tokenizer.word_index
len(sum_word_index)

75952

In [17]:
sum_vocab = {}
counter = 0
for word in sum_word_index.keys():
    if sum_word_index[word] == 0:
        print("found 0!")
        break
    if sum_word_index[word] > VOCAB_SIZE:
        continue
    else:
        sum_vocab[word] = sum_word_index[word]
        counter += 1

In [18]:
MAX_LEN = 400
pad_article = pad_sequences(article_sequences, maxlen=MAX_LEN, padding='post', truncating='post')

In [19]:
MAX_LEN_S= 100
pad_highlight = pad_sequences(summary_sequences, maxlen=MAX_LEN_S, padding='post', truncating='post')

In [20]:
pad_article.shape

(10000, 400)

In [21]:
embeddings_index = {}
with open('/content/drive/MyDrive/Colab Notebooks/glove.6B.200d.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()

In [24]:
def embedding_matrix_creater(embedding_dimention, word_index):
    embedding_matrix = np.zeros((len(word_index) + 1, embedding_dimention))
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
          # words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector
    return embedding_matrix

In [25]:
art_embedding_matrix = embedding_matrix_creater(200, word_index=art_vocab)
art_embedding_matrix.shape

(30000, 200)

In [35]:
df=pd.DataFrame(art_embedding_matrix)
df.to_excel("/content/drive/MyDrive/Colab Notebooks/trained/art_embedding_matrix.xlsx")


In [26]:
sum_embedding_matrix = embedding_matrix_creater(200, word_index=sum_vocab)
sum_embedding_matrix.shape

(30000, 200)

In [27]:
art_embedding_matrix[10]

array([-0.39374   ,  0.55684   , -0.35848001, -0.67074001,  0.073665  ,
        0.12643   , -0.013077  , -0.25279999, -0.04863   ,  0.087937  ,
       -0.20308   ,  0.43673   ,  0.18453   , -0.027372  ,  0.18987   ,
        0.23822001, -0.092781  ,  0.31736001,  0.15368   ,  0.14471   ,
        0.37138   ,  3.15409994,  0.13773   ,  0.0037497 ,  0.14195   ,
        0.40222001, -0.56691003,  0.17663001, -0.37059999,  0.19401   ,
        0.074044  , -0.19639   ,  0.093499  ,  0.78948998,  0.10651   ,
       -0.54262   , -0.82392001, -0.59018999, -0.0074686 ,  0.048896  ,
        0.26594999, -0.071857  ,  0.28968   ,  0.46733001,  0.17434999,
        0.65547001,  0.069246  , -0.78889   ,  0.20286   ,  0.44911   ,
       -0.54470998,  0.10789   ,  0.040975  ,  0.60115999, -0.14781   ,
       -0.21917   ,  0.16495   ,  0.40391001,  0.33915001,  0.36809999,
        0.034313  , -0.19927999, -0.40278   , -0.15649   ,  0.010548  ,
       -0.27677   ,  0.066898  , -0.16745999,  0.1288    , -0.11

In [28]:
def embedding_layer_creater(VOCAB_SIZE, EMBEDDING_DIM, MAX_LEN, embedding_matrix):
  embedding_layer = tf.keras.layers.Embedding(input_dim = VOCAB_SIZE, 
                                    output_dim = EMBEDDING_DIM,
                                    input_length = MAX_LEN,
                                    weights = [embedding_matrix],
                                    trainable = False)
  
  return embedding_layer



In [29]:
encoder_embedding_layer = embedding_layer_creater(VOCAB_SIZE,200, MAX_LEN, art_embedding_matrix)

In [30]:
decoder_embedding_layer = embedding_layer_creater(VOCAB_SIZE,200, MAX_LEN_S, sum_embedding_matrix)

In [ ]:
if not os.path.exists("/content/drive/MyDrive/Colab Notebooks/trained"):
    os.makedirs("/content/drive/MyDrive/Colab Notebooks/trained")

In [ ]:
!pip install chart-studio

     |████████████████████████████████| 64 kB 1.4 MB/s 
  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11447 sha256=6a53816442625a97c2047a469848b10c568d44a4bce07e5e1a653a2dd8a7071c
  Stored in directory: /root/.cache/pip/wheels/f9/8d/8d/f6af3f7f9eea3553bc2fe6d53e4b287dad18b06a861ac56ddf
Successfully built retrying


In [ ]:
from numpy.random import seed
seed(1)

from sklearn.model_selection import train_test_split
import logging

import chart_studio.plotly as py
import plotly.graph_objs as go
import matplotlib.pyplot as plt
import pandas as pd
import pydot


import keras
from keras import backend as k
k.set_learning_phase(1)
from keras.preprocessing.text import Tokenizer
from keras import initializers
from tensorflow.keras.optimizers import RMSprop
from keras.models import Sequential,Model
from keras.layers import Dense,LSTM,Dropout,Input,Activation,Add,concatenate, Embedding, RepeatVector
from keras.layers.advanced_activations import LeakyReLU,PReLU
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from tensorflow.keras.optimizers import Adam

/usr/local/lib/python3.7/dist-packages/keras/backend.py:450: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn('`tf.keras.backend.set_learning_phase` is deprecated and '


In [ ]:
from keras.layers import TimeDistributed


In [1]:
# Hyperparams

MAX_LEN = 400
VOCAB_SIZE=30000
EMBEDDING_DIM = 200
HIDDEN_UNITS = 256
VOCAB_SIZE = VOCAB_SIZE + 1

LEARNING_RATE = 0.002
BATCH_SIZE = 32
EPOCHS = 10